# Project III: Classical Non-linear Models and Police use of force
This notebook uses the Police Public Contact Survey (PPCS) dataset: `ppcs_cc.csv`.

In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import probit as probit
import logit as logit
import estimation as est
from scipy.stats import norm
from scipy.stats import t

### Load the data 

In [3]:
# Load the dataset
dat = pd.read_csv('ppcs_cc.csv')

# Inspect distribution of the target variable
print("\nDistribution of 'anyuseofforce_coded':")
print(dat['anyuseofforce_coded'].value_counts(normalize=True))

# Inspect value counts for categorical variables
categorical_vars = ["sblack", "shisp", "swhite", "sother", "smale", "omajblack", 
                    "omajhisp", "omajwhite", "omajother", "osplit", "inctype_lin", "sbehavior"]

for var in categorical_vars:
    print(f"\nValue Counts for {var}:")
    print(dat[var].value_counts())


Distribution of 'anyuseofforce_coded':
anyuseofforce_coded
0    0.994999
1    0.005001
Name: proportion, dtype: float64

Value Counts for sblack:
sblack
0    3379
1     420
Name: count, dtype: int64

Value Counts for shisp:
shisp
0    3413
1     386
Name: count, dtype: int64

Value Counts for swhite:
swhite
1    2808
0     991
Name: count, dtype: int64

Value Counts for sother:
sother
0    3614
1     185
Name: count, dtype: int64

Value Counts for smale:
smale
1    2012
0    1787
Name: count, dtype: int64

Value Counts for omajblack:
omajblack
0    3568
1     231
Name: count, dtype: int64

Value Counts for omajhisp:
omajhisp
0    3708
1      91
Name: count, dtype: int64

Value Counts for omajwhite:
omajwhite
1    3433
0     366
Name: count, dtype: int64

Value Counts for omajother:
omajother
0    3755
1      44
Name: count, dtype: int64

Value Counts for osplit:
osplit
0    3799
Name: count, dtype: int64

Value Counts for inctype_lin:
inctype_lin
2    3641
1     158
Name: count, dtype

Table with summary statistics

In [7]:
# Define groups for demographic categories
group_vars = ["swhite", "sblack", "shisp", "sother"]

# List of all variables for which we want to compute means
all_vars = dat.columns

# Initialize an empty DataFrame to store results
summary_table = pd.DataFrame()

# Calculate the overall mean for each variable
overall_means = dat[all_vars].mean()
summary_table["Variable"] = all_vars
summary_table["Full Sample"] = overall_means.values

# Calculate the mean for each variable within each group
for group in group_vars:
    group_means = dat.loc[dat[group] == 1, all_vars].mean()
    summary_table[group.capitalize()] = group_means.values

# Add a row for "Number of Observations"
num_obs_row = pd.DataFrame({
    "Variable": ["Number of Observations"],
    "Full Sample": [dat.shape[0]],
    **{group.capitalize(): [dat.loc[dat[group] == 1].shape[0]] for group in group_vars}
})

# Append the "Number of Observations" row to the summary table
summary_table = pd.concat([summary_table, num_obs_row], ignore_index=True)

# Format the table for display
summary_table = summary_table.set_index("Variable")
print(summary_table)

# Optional: Save the table to a CSV for further analysis
summary_table.to_csv('grouped_summary_statistics_with_observations.csv')


                        Full Sample       Swhite       Sblack        Shisp  \
Variable                                                                     
sblack                     0.110555     0.000000     1.000000     0.000000   
shisp                      0.101606     0.000000     0.000000     1.000000   
swhite                     0.739142     1.000000     0.000000     0.000000   
sother                     0.048697     0.000000     0.000000     0.000000   
smale                      0.529613     0.521368     0.519048     0.585492   
sage                      41.010003    42.147792    39.183333    36.225389   
sempl                      0.695446     0.699786     0.688095     0.676166   
sincome                    2.164780     2.218305     1.935714     1.997409   
spop                       1.362727     1.271011     1.657143     1.652850   
daytime                    0.666491     0.680912     0.621429     0.642487   
inctype_lin                1.958410     1.957621     1.966667   

In [33]:
# Declare labels    
y_lab = 'anyuseofforce_coded'
#x_lab = ['const', 'sblack', 'shisp', 'sother']
#x_lab = ['const', 'sblack', 'shisp', 'sother', 'smale', 'sempl', 'sincome', 'spop', 'sage', 'sagesq']
#x_lab = ['const', 'sblack', 'shisp', 'sother', 'smale', 'sempl', 'sincome', 'spop', 'sage', 'sagesq', 'daytime', 'inctype_lin', 'omajblack', 'omajhisp', 'omajother']
x_lab = ['const', 'sblack', 'shisp', 'sother', 'smale', 'sempl', 'sincome', 'spop', 'sage', 'sagesq', 'daytime', 'inctype_lin', 'omajblack', 'omajhisp', 'omajother', 'sbehavior']

dat['sage'] = dat['sage'] / 10
dat['sagesq'] = dat.sage * dat.sage 

# create extra variables 
N = dat.shape[0]
dat['const'] = np.ones((N,))

# Rebuild the dataset without 'osplit'
dat = dat[[y_lab] + x_lab].copy()

# Check for missing data
assert dat.notnull().all(axis=1).all(), 'Missing values detected. Clean your data!'

dat.tail(5)

,anyuseofforce_coded,const,sblack,shisp,sother,smale,sempl,sincome,spop,sage,sagesq,daytime,inctype_lin,omajblack,omajhisp,omajother,sbehavior
3794,0,1.0,0,0,0,0,1,3,1,7.2,51.84,0,1,0,0,0,1
3795,0,1.0,0,0,0,0,0,2,1,7.1,50.41,1,2,0,0,0,0
3796,0,1.0,0,0,0,0,0,1,1,7.6,57.76,1,2,0,0,0,0
3797,0,1.0,0,0,0,0,0,3,4,7.9,62.41,1,2,0,0,0,0
3798,0,1.0,0,0,0,0,0,2,1,7.5,56.25,1,2,0,0,0,0


In [34]:
# Extract y and X
y = dat[y_lab].values
x = dat[x_lab].values
K = x.shape[1]

print(K)
print(np.shape(x))

16
(3799, 16)


In [35]:
count_violent_1 = (dat['anyuseofforce_coded'] == 1).sum()
print(f"Number of 1s in 'anyuseofforce_coded': {count_violent_1}")

Number of 1s in 'anyuseofforce_coded': 19


## Estimate using Probit

In [36]:
# Initialize starting values
theta0 = probit.starting_values(y, x)

# Display starting values
print("Starting values for theta:", theta0)

probit_results = est.estimate(probit.q, theta0, y, x, cov_type='Sandwich')

Starting values for theta: [ 0.13618175  0.00493452  0.02218758 -0.00094007  0.01082135 -0.0155645
  0.00220925  0.01026053  0.00248039 -0.00072741 -0.00447688 -0.07037371
 -0.01365712 -0.00313071 -0.01591562  0.08840108]
Optimization terminated successfully.
         Current function value: 0.022253
         Iterations: 157
         Function evaluations: 2941
         Gradient evaluations: 173


In [37]:
probit_tab = est.print_table(x_lab, probit_results, title=f'Probit, y = {y_lab}')
probit_tab

Optimizer succeded after 157 iter. (2941 func. evals.). Final criterion:  0.02225.
Probit, y = anyuseofforce_coded


,theta,se,t
const,-2.6163,0.6954,-3.7625
sblack,0.2036,0.2991,0.6808
shisp,0.4164,0.2382,1.7481
sother,0.1043,0.4412,0.2364
smale,0.5597,0.2096,2.6700
sempl,-0.4997,0.2205,-2.2667
sincome,0.1021,0.1244,0.8204
spop,0.1988,0.0746,2.6642
sage,0.4859,0.3718,1.3067
sagesq,-0.0790,0.0498,-1.5852


## Estimate using Logit

In [38]:
logit_results = est.estimate(logit.q, theta0, y, x, cov_type='Sandwich')

Optimization terminated successfully.
         Current function value: 0.022389
         Iterations: 201
         Function evaluations: 3604
         Gradient evaluations: 212


In [39]:
logit_tab = est.print_table(x_lab, logit_results, title=f'Logit, y = {y_lab}')
logit_tab

Optimizer succeded after 201 iter. (3604 func. evals.). Final criterion:  0.02239.
Logit, y = anyuseofforce_coded


,theta,se,t
const,-5.6081,1.9346,-2.8988
sblack,0.4375,0.8073,0.5419
shisp,0.9406,0.6120,1.5371
sother,-0.0904,1.2050,-0.0750
smale,1.1537,0.5733,2.0123
sempl,-1.1621,0.6152,-1.8889
sincome,0.2021,0.3421,0.5909
spop,0.4811,0.1888,2.5490
sage,1.3022,1.1762,1.1071
sagesq,-0.2206,0.1652,-1.3357


## Average partial effects

### Probit

In [40]:
# Estimating the average partial effects 
indices = [x_lab.index('sblack'), x_lab.index('shisp'), x_lab.index('sother')]  
labels = ['sblack', 'shispanic', 'sother'] 
probit.properties(x, probit_results['theta'],print_out = True,se=True,indices=indices, labels = labels)

,Estimate
sblack,0.002
shispanic,0.006
sother,0.001


### Logit

In [41]:
# Estimating the average partial effects 
indices = [x_lab.index('sblack'), x_lab.index('shisp'), x_lab.index('sother')]  
labels = ['sblack', 'shispanic', 'sother']  
logit.properties(x, logit_results['theta'],print_out = True,se=True,indices=indices, labels = labels)

,Estimate
sblack,0.002
shispanic,0.005
sother,-0.000


## Partial Effects

#### Defining different fixed vectors

In [42]:
#means of the regressors
print(f"{np.mean(dat['sage']):.2f}")
print(f"{np.mean(dat['sagesq']):.2f}")
print(f"{np.mean(dat['sincome']):.2f}")
print(f"{np.mean(dat['spop']):.2f}")

4.10
19.42
2.16
1.36


In [43]:
###ORIGINAL VECTOR###
# Let us make a vector of the values we want to investigate
x_lab = ['const', 'sblack', 'shisp', 'sother', 'smale', 'sage', 'sempl', 'sincome', 'spop', 'daytime', 'inctype_lin', 'omajblack', 'omajhisp', 'omajother', 'sbehavior','sagesq']

x_me = np.array([1, 0, 0, 0, 1, 4.10, 0, 2.16, 1.36,0,1,0,0,0, 0,19.42]).reshape(1, -1)
pd.DataFrame(x_me, columns=x_lab, index=['x_me'])


,const,sblack,shisp,sother,smale,sage,sempl,sincome,spop,daytime,inctype_lin,omajblack,omajhisp,omajother,sbehavior,sagesq
x_me,1.0,0.0,0.0,0.0,1.0,4.1,0.0,2.16,1.36,0.0,1.0,0.0,0.0,0.0,0.0,19.42


In [44]:
### BEHAVIOR = 1 ###
# Let us make a vector of the values we want to investigate
#x_me= np.array([1, 0, 0, 0, 1, 4.1, 0, 2.16, 1.36,0,1,0,0,0,1,19.42]).reshape(1, -1)
#pd.DataFrame(x_me, columns=x_lab, index=['x_behavior'])


In [45]:
###DAYTIME = 1###
# Let us make a vector of the values we want to investigate
#x_me = np.array([1, 0, 0, 0, 1, 4.1, 0, 2.16, 1.36,1,1,0,0,0,0,19.42]).reshape(1, -1)
#pd.DataFrame(x_me, columns=x_lab, index=['x_daytime'])


#### Swiching race from white to black, hispanic and other

In [46]:
#k=1: black 
#k=2: hispanic
#k=3: other

k = 1
x_me2 = x_me.copy()
x_me2[:, k] = 1  # Keep everythin the same, but change foreign to 1 for all obs. 
pd.DataFrame(x_me2, columns=x_lab, index=['x_me2'])

,const,sblack,shisp,sother,smale,sage,sempl,sincome,spop,daytime,inctype_lin,omajblack,omajhisp,omajother,sbehavior,sagesq
x_me2,1.0,1.0,0.0,0.0,1.0,4.1,0.0,2.16,1.36,0.0,1.0,0.0,0.0,0.0,0.0,19.42


### Probit

In [47]:
b_pr = probit_tab.theta.values
me_race_pr = probit.G(x_me2@b_pr) - probit.G(x_me@b_pr) 

In [48]:
gx0 = norm.pdf(x_me@b_pr)
gx2 = norm.pdf(x_me2@b_pr)

grad_d_pr = gx2*x_me2 - gx0*x_me

In [49]:
def get_se(grad, cov):
    cov_me = grad@cov@grad.T
    return np.sqrt(np.diag(cov_me))

se_d_pr = get_se(grad_d_pr, probit_results['cov'])

In [50]:
me_dict = {'Marginal Effect': me_race_pr[0],
           's.e.':            se_d_pr}
tab = pd.DataFrame(me_dict)
tab['t'] = tab['Marginal Effect'] / tab['s.e.']
tab.index.name = 'Var'
tab.round(6)

,Marginal Effect,s.e.,t
Var,,,
0,0.0,0.0,0.0


### Logit

In [51]:
b_lg = logit_tab.theta.values
me_race_lg = logit.G(x_me2@b_lg) - logit.G(x_me@b_lg)

In [57]:
# Compute the logistic function exponential terms for x_me2 and x_me
exp_x0_b = np.exp(-np.dot(x_me, b_lg))
exp_x2_b = np.exp(-np.dot(x_me2, b_lg))

grad_d_lg = (x_me2 * exp_x2_b)/ (1 + exp_x2_b)**2 - (x_me * exp_x0_b)/ (1 + exp_x0_b)**2

se_d_lg = get_se(grad_d_lg, logit_results['cov'])

In [54]:
def race_lg(b_lg):
    race_logit = logit.G(x_me2@b_lg) - logit.G(x_me@b_lg)
    return race_logit

qq = lambda b_lg: race_lg(b_lg)
grad_d_lg = est.centered_grad(qq,b_lg)

def get_se(grad, cov):
    cov_me = grad@cov@grad.T
    return np.sqrt(np.diag(cov_me))

se_d_lg = get_se(grad_d_lg, logit_results['cov'])

In [58]:
me_dict = {'Marginal Effect': me_race_lg[0],
           's.e.':            se_d_lg}
tab = pd.DataFrame(me_dict)
tab['t'] = tab['Marginal Effect'] / tab['s.e.']
tab.index.name = 'Var'
tab.round(6)

,Marginal Effect,s.e.,t
Var,,,
0,0.0,0.0,0.0
